# Machine Learning approach Part-of-Speech-Tagging with Hidden Markov Models

In this notebook, you'll use the [Pomegranate](https://pomegranate.readthedocs.io/en/latest/) library to build a hidden Markov model for part of speech tagging using a "universal" tagset. Hidden Markov models have been able to achieve >96% tag accuracy with larger tagsets on realistic text corpora. Hidden Markov models have also been used for speech recognition and speech generation, machine translation, gene recognition for bioinformatics, and human gesture recognition for computer vision, and more.

# Library

# Pomegranate Installation

!conda install -n ghaluh pomegranate -y

if you don't have environment, following these options
1. Install Pomegranate without Conda: pip install pomegranate
2. Create a Conda Environment: 
#conda create -n my_env python=3.8
#conda activate my_env
#conda install pomegranate
4. Use an Existing Environment: 
#conda activate existing_env
#conda install pomegranate
5. Virtual Environments: 
#python -m venv my_env
#source my_env/bin/activate   # On Windows, use `my_env\Scripts\activate`
#pip install pomegranate

# Read & preprocess the dataset

We'll start by reading in a text corpus and splitting it into a training and testing dataset. The data set is a copy of the [Brown corpus](https://en.wikipedia.org/wiki/Brown_Corpus) (originally from the [NLTK](https://www.nltk.org/) library) that has already been pre-processed to only include the [universal](chrome-extension://efaidnbmnnnibpcajpcglclefindmkaj/https://arxiv.org/pdf/1104.2086.pdf) tagset. You should expect to get slightly higher accuracy using this simplified tagset than the same model would achieve on a larger tagset like the full [Penn treebank](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html) tagset, but the process you'll follow would be the same.

The Dataset class provided in helpers.py will read and parse the corpus. You can generate your own datasets compatible with the reader by writing them to the following format. The dataset is stored in plaintext as a collection of words and corresponding tags. Each sentence starts with a unique identifier on the first line, followed by one tab-separated word/tag pair on each following line. Sentences are separated by a single blank line.

Sentences

`Dataset.sentences` is a dictionary of all sentences in the training corpus, each keyed to a unique sentence identifier. Each Sentence is itself an object with two attributes: a tuple of the words in the sentence named words and a tuple of the tag corresponding to each word named tags.

**Note:** The underlying iterable sequence is **unordered** over the sentences in the corpus; it is not guaranteed to return the sentences in a consistent order between calls. Use `Dataset.stream()`, `Dataset.keys`, `Dataset.X`, or `Dataset.Y` attributes if you need ordered access to the data.

Counting Unique Elements

You can access the list of unique words (the dataset vocabulary) via Dataset.vocab and the unique list of tags via Dataset.tagset.

Accessing word and tag Sequences

The Dataset.X and Dataset.Y attributes provide access to ordered collections of matching word and tag sequences for each sentence in the dataset.

Accessing (word, tag) Samples

The Dataset.stream() method returns an iterator that chains together every pair of (word, tag) entries across all sentences in the entire corpus.

For both our baseline tagger and the HMM model we'll build, we need to estimate the frequency of tags & words from the frequency counts of observations in the training corpus. In the next several cells you will complete functions to compute the counts of several sets of counts.

# Build a Most Frequent Class tagger

Perhaps the simplest tagger (and a good baseline for tagger performance) is to simply choose the tag most frequently assigned to each word. This "most frequent class" tagger inspects each observed word in the sequence and assigns it the label that was most often assigned to that word in the corpus.

IMPLEMENTATION: Pair Counts

Complete the function below that computes the joint frequency counts for two input sequences.

IMPLEMENTATION: Most Frequent Class Tagger

Use the pair_counts() function and the training dataset to find the most frequent class label for each word in the training data, and populate the mfc_table below. The table keys should be words, and the values should be the appropriate tag string.

The MFCTagger class is provided to mock the interface of Pomegranite HMM models so that they can be used interchangeably.

# Making Predictions with a Model

The helper functions provided below interface with Pomegranate network models & the mocked MFCTagger to take advantage of the [missing value](https://pomegranate.readthedocs.io/en/latest/nan.html) functionality in Pomegranate through a simple sequence decoding function. Run these functions, then run the next cell to see some of the predictions made by the MFC tagger.

# Example Decoding Sequences with MFC Tagger

# Evaluating Model Accuracy

The function below will evaluate the accuracy of the MFC tagger on the collection of all sentences from a text corpus.

# Evaluate the accuracy of the MFC tagger

Run the next cell to evaluate the accuracy of the tagger on the training and test corpus.

# Build an HMM tagger

The HMM tagger has one hidden state for each possible tag, and parameterized by two distributions: the emission probabilties giving the conditional probability of observing a given word from each hidden state, and the transition probabilities giving the conditional probability of moving between tags during the sequence.

We will also estimate the starting probability distribution (the probability of each tag being the first tag in a sequence), and the terminal probability distribution (the probability of each tag being the last tag in a sequence).

The maximum likelihood estimate of these distributions can be calculated from the frequency counts as described in the following sections where you'll implement functions to count the frequencies, and finally build the model. The HMM model will make predictions according to the formula:

$$t^n_i=argmax_{t^n_i}∏^n_{i=1} P(w_i|t_i)P(t_i|t_{i−1})$$
 
Refer to Speech & Language Processing [Chapter 10](chrome-extension://efaidnbmnnnibpcajpcglclefindmkaj/https://web.stanford.edu/~jurafsky/slp3/10.pdf) for more information.

# IMPLEMENTATION: Unigram Counts

Complete the function below to estimate the co-occurrence frequency of each symbol over all of the input sequences. The unigram probabilities in our HMM model are estimated from the formula below, where N is the total number of samples in the input. (You only need to compute the counts for now.)
$$P(tag_1)=\frac{C(tag_1)}{N}$$

# IMPLEMENTATION: Bigram Counts

Complete the function below to estimate the co-occurrence frequency of each pair of symbols in each of the input sequences. These counts are used in the HMM model to estimate the bigram probability of two tags from the frequency counts according to the formula:
$$P(tag_2|tag_1)=\frac{C(tag_2|tag_1)}{C(tag_2)}$$

# IMPLEMENTATION: Sequence Starting Counts

Complete the code below to estimate the bigram probabilities of a sequence starting with each tag.

# IMPLEMENTATION: Sequence Ending Counts

Complete the function below to estimate the bigram probabilities of a sequence ending with each tag.

# IMPLEMENTATION: Basic HMM Tagger

Use the tag unigrams and bigrams calculated above to construct a hidden Markov tagger.

Add one state per tag:
The emission distribution at each state should be estimated with the formula:  $$P(w|t)=\frac{C(t,w)}{C(t)}$$
 
Add an edge from the starting state basic_model.start to each tag:
The transition probability should be estimated with the formula:  $$P(t|start)=\frac{C(start,t)}{C(start)}$$
 
Add an edge from each tag to the end state basic_model.end:
The transition probability should be estimated with the formula:  $$P(end|t)=\frac{C(t,end)}{C(t)}$$
 
Add an edge between every pair of tags:
The transition probability should be estimated with the formula:  $$P(t_2|t_1)=\frac{C(t1,t2)}{C(t1)}$$

# Example Decoding Sequences with the HMM Tagger